# Connecting to Elasticsearch and Inserting Data
1. First import all the releavnt modules.


In [ ]:
import requests
import json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from pprint import pprint
from datetime import datetime
print ("All Imported")

## Update required variables
2. You will need to copy your domain for the Elasticsearch you have created.

In [ ]:
# Copy your Elasticsearch domain here
domain = '' 
es = Elasticsearch(domain)
print('Variables Set')

## Functions 
3. This section contains functions that are used in the following code blocks. 

In [ ]:
def return_indices(var_domain, var_area):
    res = requests.get(domain + area, params=param)
    print ('\nHTTP code',res.status_code, '--response:', res, '\n')
    print (res.text)

print("Functions defined")

## Status of your Domain
4. There are a number of different ways to return the results. I have found print(res.test) to be the best for this result.
There are a few options commented out which will return the results in a different format.

In [ ]:
res = requests.get(domain)
##print (res.content)
##pprint(res.text)
print(res.text)

## Node and Index Status
5. Returns the status of the node and returns details of any existing indices.

In [ ]:
res = requests.get(domain + '/_cat/nodes?v&pretty')
##print (res.content)
pprint(res.text)

param = (('v',''),) # '-v is for verbose
area = '/_cat/indices' # /_cat/indices, /_cat/nodes
return_indices(domain, area)

# Creating an Index and Inserting Data
6. It is possible to work with index's without inserting a document. The below code creates an empty index.

In [ ]:
index_name = 'test_index'
es.indices.create(index=index_name)
return_indices(domain, area)

7. Deletes and checks index removal.


In [ ]:
index_name = 'test_index'
es.indices.delete(index=index_name)
return_indices(domain, area)

## Creating Index and Inserting Data
8. The below will create or update an existing index and add the single json document to it.

In [ ]:
document = {
    'author':'kimchy',
    'text':'Elasticsearch: cool. bonsai cool',
    'timestamp': datetime.now(),
}
index_name = 'test_index'
doc_type = 'test'

res = es.index(index=index_name, doc_type=doc_type, id=1, body=document)
print(res['result'])

return_indices(domain, area)

# Searching your Index
9. The below is a very basic search which will return every document within the index.

In [ ]:
index_name = 'test_index'
es.search(index=index_name)

10. You can add additional search parameters to the command, but with only one document this is a little pointless. 

In [ ]:
doc_type = "test"
es.search(index=index_name, doc_type=doc_type)

# Loading a larger dataset
To show the real power of search both in this notebook and in Kibana you need a larger dataset.
I have download a sample json file containing multiply docuements from :
This data being uploaded contains 6,000 account records.

11. Set the directory containing the file to upload.
Note that the backslashs have been replaced with forward slashs.

In [ ]:
# add your local directory here, remember that you need to use / in the directory path
directory = "" 
index_name = "accounts"
doc_type = "finance"
print("Variables set")

12. This next code block users the Elasticserach helper command to bulk uploaded the contents of the accounts json file.

In [ ]:

op_list = []
with open(directory) as json_file:
    for record in json_file:
        op_list.append({
                'op_type':'index',
                '_index': index_name,
                '_type': doc_type,
                '_source': record
        })
helpers.bulk(client=es, actions=op_list)

print("List Uploaded")


In [ ]:
return_indices(domain, area)

# Searching an Index
13. The two below code bocks are different ways to search the created index to return the documents. 

In [ ]:

start_from = 0 # this determines from which data item the data should be returned
size = 2    # this is the number of docuemnts to be returned
sort = {'balance':{'order':'asc'}}
body = {
    "from":start_from,
    "size":size,
    "fields": ["balance"],
    "query": {
        "term": {
        }
    },
    "sort": {
        "balance": {
            "order": "asc"
        }
    }
}
res = es.search(index=index_name, 
                doc_type=doc_type, 
                size=size,
                sort=sort)
res

In [ ]:
req = []
start_from = 0
size = 1    # this is the number of docuemnts to be returned
req_head = {'index': index_name, 'type': doc_type}
req_body = {
    'query': {'range': {'balance': {'gte':10000, 'lt': 20000}}},
    'from': start_from, 'size': size,
}
req.extend([req_head, req_body])

res = es.msearch(body = req)
res